# Multiple Classification

In this exercise, will train a custome neural networks model that will accurately predict a sort of beer based on some evaluation criterias such as look, aroma, palate or flavor. We will be woking on the dataset: https://drive.google.com/file/d/1vYyJL_IB6KjKCxuk9kg4vIMPGTtoX8Ek/view


The steps are:
1. Setup Repository
2. Load and Explore data
3. Prepare Data
4. Evaluation
5. Baseline Model
6. Define Architecture
7. Train Model
8. Push Changes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg') 
%matplotlib inline

## 1. Setup Repository

**[1.1]** Go to the folder I created previously `adv_dsi_ass_1`

In [2]:
cd /Users/frodo/Downloads/36114/project/adv_dsi_2023/adv_dsi_ass_2/ass_2

/Users/frodo/Downloads/36114/project/adv_dsi_2023/adv_dsi_ass_2/ass_2


**[1.2]** Copy the cookiecutter data science template

In [ ]:
cookiecutter -c v1 https://github.com/drivendata/cookiecutter-data-science

**[1.3]** Create a file called `Dockerfile` and add the following content:

In [ ]:
touch Dockerfile

In [ ]:
FROM jupyter/scipy-notebook:python-3.8.13
RUN pip install torch torchvision torchtext --extra-index-url https://download.pytorch.org/whl/cpu
ENV PYTHONPATH "${PYTHONPATH}:/home/jovyan/work"
WORKDIR /home/jovyan/work

**[1.4]** Build the image from this Dockerfile

In [ ]:
docker build -t pytorch-notebook:latest .

**[1.5]** Run the built Docker image

In [ ]:
docker run  -dit --rm --name adv_dsi_2023 -p 8888:8888 -e JUPYTER_ENABLE_LAB=yes -v ~/Projects/adv_dsi_2023/adv_dsi_ass_2:/home/jovyan/work -v ~/Projects/adv_dsi_2023/src:/home/jovyan/work/src pytorch-notebook:latest 

**[1.6]** Display last 50 lines of logs

In [ ]:
docker logs --tail 50 adv_dsi_ass_2

**[1.7]** Initialise the repo

In [ ]:
git init

**[1.8]** In the local repo `adv_dsi_ass_2`,

In [ ]:
# Solution
git remote add origin git@github.com:frodorocky/adv_dsi_ass_2.git

**[1.9]** Add changes to git staging area and commit them

In [ ]:
# Solution
git add .
git commit -m "init"

**[1.10]** Push master branch to origin

In [ ]:
git push --set-upstream origin master

**[1.11]** Create a new git branch called `pytorch_reg`

In [ ]:
git checkout -b pytorch_reg

## 2. Load and explore data

We will do data cleaning and data exploration simultaneously because data exploration often goes hand in hand with data cleaning processes in our experiment

**[2.1]** Load the data in a dataframe called `df`

In [3]:
df = pd.read_csv('data/raw/beer_reviews.csv')

**[2.2]** Display the first 5 rows of df

In [4]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


**[2.3]** Set the index of a DataFrame to the values of the column 'Id'.

In [5]:
df = df.set_index(['beer_beerid','review_time'])

**[2.3]** Display the shape of train data

In [6]:
df.shape

(1586614, 11)

**[2.4]** Display the summary (info) of df

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1586614 entries, (47986, 1234817823) to (33061, 1160005319)
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_overall      1586614 non-null  float64
 3   review_aroma        1586614 non-null  float64
 4   review_appearance   1586614 non-null  float64
 5   review_profilename  1586266 non-null  object 
 6   beer_style          1586614 non-null  object 
 7   review_palate       1586614 non-null  float64
 8   review_taste        1586614 non-null  float64
 9   beer_name           1586614 non-null  object 
 10  beer_abv            1518829 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 192.1+ MB


**[2.5]** Display the descriptive statistics of df

In [8]:
df.describe()

,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06
mean,3.130099e+03,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00
std,5.578104e+03,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02
25%,1.430000e+02,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00
50%,4.290000e+02,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00
75%,2.372000e+03,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00
max,2.800300e+04,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01


**[2.6]** Distribution of Y

In [9]:
df.beer_style.value_counts()

American IPA                        117586
American Double / Imperial IPA       85977
American Pale Ale (APA)              63469
Russian Imperial Stout               54129
American Double / Imperial Stout     50705
                                     ...  
Gose                                   686
Faro                                   609
Roggenbier                             466
Kvass                                  297
Happoshu                               241
Name: beer_style, Length: 104, dtype: int64

**[2.7]** Distribution of X

In [10]:
for column in df.columns:
    print(f"\nFeature: {column}")
    if df[column].dtype in ['int64', 'float64']:
        print(df[column].describe())
    else:
        print(df[column].value_counts())


Feature: brewery_id
count    1.586614e+06
mean     3.130099e+03
std      5.578104e+03
min      1.000000e+00
25%      1.430000e+02
50%      4.290000e+02
75%      2.372000e+03
max      2.800300e+04
Name: brewery_id, dtype: float64

Feature: brewery_name
Boston Beer Company (Samuel Adams)    39444
Dogfish Head Brewery                  33839
Stone Brewing Co.                     33066
Sierra Nevada Brewing Co.             28751
Bell's Brewery, Inc.                  25191
                                      ...  
Brauerei Stolz GmbH & Co. KG              1
Hausbrauerei Düll                         1
Browar Grybów                             1
Staro&#269;eský Pivovárek Dobruka        1
Spire Brewery                             1
Name: brewery_name, Length: 5742, dtype: int64

Feature: review_overall
count    1.586614e+06
mean     3.815581e+00
std      7.206219e-01
min      0.000000e+00
25%      3.500000e+00
50%      4.000000e+00
75%      4.500000e+00
max      5.000000e+00
Name: review_ov

## 3. Prepare data

**[3.1]** Create a copy of `df` and save it into a variable called `df_cleaned`

In [11]:
df_cleaned = df.copy()

**[3.2]** Dealing missing value

In [12]:
df_cleaned = df_cleaned.dropna()

**[3.3]** Extract the target variable into a variable called y

In [14]:
y = df_cleaned.pop('beer_style')

**[3.4]** Mapping labels to numbers

In [15]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

**[3.5]** Define customer dataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

In [17]:
class MyDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __getitem__(self, index):

        single_data = torch.from_numpy(self.data[index].todense()).float()
        single_target = torch.tensor(self.target[index]).long()
        return single_data, single_target

    def __len__(self):
        return self.data.shape[0]

**[3.6]** Define preprocessor to deal with numeric features and categorical features

In [18]:
num_cols = df_cleaned.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns
cat_cols = df_cleaned.select_dtypes(include=['object', 'category']).columns

In [19]:
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

In [20]:
data_preprocessed = preprocessor.fit_transform(df_cleaned)

**[3.7]** split dataset

In [21]:
# Convert DataFrames/Series to numpy arrays before splitting
X_trainval, X_test, y_trainval, y_test = train_test_split(data_preprocessed, y_encoded, test_size=0.2, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, stratify=y_trainval) 

**[3.8]** Dataset Load

In [26]:
train_dataset = MyDataset(X_train, y_train)
val_dataset = MyDataset(X_val, y_val)
test_dataset = MyDataset(X_test, y_test)

In [27]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 4. Train model

**[4.1]** Initialize hyperparameter

In [ ]:
import torch
from torch import nn
from torch.optim import Adam

# set hyperparameter
input_size = X_train.shape[1]  
hidden_size = 64  
num_classes = len(np.unique(y_train))  
num_epochs = 50  
learning_rate = 0.01  

**[4.2]** define model

In [ ]:
# define model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = out.squeeze()
        return out

**[4.3]** train model

In [40]:
# create model
model = MLP(input_size, hidden_size, num_classes)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

best_loss = float('inf')

# train model
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        features = features.float()
        labels = labels.long()
        
        if torch.cuda.is_available():
            features = features.cuda()
            labels = labels.cuda()
        
        # Forward Propagation
        outputs = model(features)
        loss = criterion(outputs, labels)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
    # validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for i, (features, labels) in enumerate(val_loader):
            features = features.float()
            labels = labels.long()
            
            if torch.cuda.is_available():
                features = features.cuda()
                labels = labels.cuda()

            outputs = model(features)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
        val_loss /= len(val_loader)

    # save best model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'models/best_model.pth')

    print('Epoch [{}/{}], Loss: {:.4f}, Val Loss: {:.4f}'
          .format(epoch+1, num_epochs, loss.item(), val_loss))


Epoch [1/50], Loss: 3.0468, Val Loss: 3.1885
Epoch [2/50], Loss: 2.4736, Val Loss: 2.6170
Epoch [3/50], Loss: 2.0019, Val Loss: 2.0218
Epoch [4/50], Loss: 1.6577, Val Loss: 1.5755
Epoch [5/50], Loss: 1.3570, Val Loss: 1.2647
Epoch [6/50], Loss: 1.0844, Val Loss: 1.0440
Epoch [7/50], Loss: 0.9218, Val Loss: 0.8821
Epoch [8/50], Loss: 0.7825, Val Loss: 0.7618
Epoch [9/50], Loss: 0.7041, Val Loss: 0.6686
Epoch [10/50], Loss: 0.5943, Val Loss: 0.5963
Epoch [11/50], Loss: 0.5528, Val Loss: 0.5362
Epoch [12/50], Loss: 0.4640, Val Loss: 0.4890
Epoch [13/50], Loss: 0.4095, Val Loss: 0.4509
Epoch [14/50], Loss: 0.3528, Val Loss: 0.4173
Epoch [15/50], Loss: 0.2985, Val Loss: 0.3876
Epoch [16/50], Loss: 0.2742, Val Loss: 0.3628
Epoch [17/50], Loss: 0.2317, Val Loss: 0.3406
Epoch [18/50], Loss: 0.2004, Val Loss: 0.3219
Epoch [19/50], Loss: 0.1648, Val Loss: 0.3065
Epoch [20/50], Loss: 0.1407, Val Loss: 0.2905
Epoch [21/50], Loss: 0.1173, Val Loss: 0.2768
Epoch [22/50], Loss: 0.0947, Val Loss: 0.26

## 4. Evaluation on test data

**[4.1]** we applied the trained model on test data to obtain model performance 

In [41]:
from sklearn.metrics import classification_report
import numpy as np

# load best model
model.load_state_dict(torch.load('models/best_model.pth'))

# predict and ground truth
y_pred = []
y_true = []

with torch.no_grad():
    for features, labels in test_loader:
        features = features.float()
        labels = labels.long()
        
        if torch.cuda.is_available():
            features = features.cuda()
            labels = labels.cuda()

        # prediction
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)

        y_pred.extend(predicted.cpu().numpy().tolist())
        y_true.extend(labels.cpu().numpy().tolist())

**[4.2]** Evaluation

In [42]:
# compute recision, recall, and f1 score
print(classification_report(y_true, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1444
           1       0.98      0.99      0.98      5793
           2       0.96      0.98      0.97      8346
           3       0.99      0.97      0.98      1722
           4       0.98      0.99      0.98      5253
           5       0.99      0.97      0.98      2232
           6       0.91      0.94      0.92      2160
           7       0.98      0.97      0.98      4826
           8       0.95      0.87      0.91       239
           9       0.99      0.99      0.99     17021
          10       0.98      0.97      0.97      1079
          11       0.99      0.99      0.99     10027
          12       0.98      0.99      0.99     22629
          13       0.99      0.97      0.98       766
          14       0.97      0.98      0.97     11614
          15       0.97      0.96      0.97      1673
          16       0.98      0.97      0.98      4578
          17       0.98    

## 5. Push changes

**[5.1]** Add the changes to git staging area

In [ ]:
git add .

**[5.2]** Create the snapshot of the repository and add a description

In [ ]:
git commit -m "MLP"

**[5.3]** Push snapshot to Github

In [ ]:
git push --set-upstream origin main

**[5.4]** Check out to the branch

In [ ]:
git checkout main

**[5.5]** Pull the latest updates

In [ ]:
git pull

**[5.6]** Stop the Docker container

In [ ]:
docker stop adv_dsi_ass_2

## Reference

So, A. (2023). 36114_AdvDSI_Lab3_Exercise3_Solutions. https://drive.google.com/file/d/17UpMxkj4slAQ2BDyAeNEYzUfTA9Ww3x6/view